In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# 파일 경로
file_crime = '경찰청_범죄 발생 지역별 통계_20231231.csv'
file_population = '202411_202411_연령별인구현황_월간.csv'
file_station = '경찰청_경찰관서위치정보(지구대 파출소포함)_20230630.csv'
file_cctv = '인천광역시_방범 CCTV  설치현황_20240630.csv'
file_streetlight = '인천광역시_도로조명시설 설치현황_20231231.csv'

# 파일 읽기
crime_data = pd.read_csv(file_crime, encoding='euc-kr')
population_data = pd.read_csv(file_population, encoding='euc-kr')
station_data = pd.read_csv(file_station, encoding='euc-kr')
cctv_data = pd.read_csv(file_cctv, encoding='euc-kr')
streetlight_data = pd.read_csv(file_streetlight, encoding='euc-kr')

# 1. 관서별 총 범죄량 계산
# 선택할 열 이름 정의
incheon_columns = [
    '인천중구', '인천동구', '인천미추홀구', '인천연수구', 
    '인천남동구', '인천부평구', '인천계양구', '인천서구', 
    '인천강화군', '인천옹진군'
]

crime_data_filtered = crime_data[crime_data['범죄대분류'].isin(['강력범죄', '절도범죄', '폭력범죄'])]
incheon_crime_data = crime_data_filtered[incheon_columns]
crime_totals = incheon_crime_data.sum().reset_index() # 구분, 발생검거 제외하고 합한뒤 인덱스를 열로 변환 
crime_totals.columns = ['지역구', '총범죄량']

# 관서 -> 행정구역 매핑
crime_to_region_map = {
    '인천중구': '중구',
    '인천동구': '동구',
    '인천미추홀구': '미추홀구',
    '인천연수구': '연수구',
    '인천남동구': '남동구',
    '인천부평구': '부평구',
    '인천계양구': '계양구',
    '인천서구': '서구',
    '인천강화군': '강화군',
    '인천옹진군': '옹진군'
}

crime_totals['행정구역'] = crime_totals['지역구'].map(crime_to_region_map)

# 2. 행정구역별 총 인구수 계산
population_data['총인구수'] = population_data['2024년11월_계_총인구수'].str.replace(',', '').astype(int) #쉼표 뺴고, 열의 요소를 int형으로 변환

# str.extract 정규표현식을 사용해 인천광역시 뒤의 공백을 허용하고 공백이 아닌 (문자하나)를 추출함
population_data['행정구역'] = population_data['행정구역'].str.extract(r'인천광역시\s*(\S+)')
population_totals = population_data[['행정구역', '총인구수']]

# 범죄 데이터와 인구 데이터 병합 -> 인구수당 범죄율 계산
crime_rate = crime_totals.merge(population_totals, on='행정구역', how='left') #파일 병합
crime_rate['인구수당범죄율'] = crime_rate['총범죄량'] / crime_rate['총인구수'] #범죄율 구하기

# 3. 자치구별 경찰서 수 계산
station_data_filtered = station_data[station_data['시도청'] == '인천청']
station_data_filtered['행정구역'] = station_data_filtered['주소'].str.extract(r'인천광역시\s*(\S+)')
# 자치구를 기준으로 경찰서 수를 계산하고 열이름을 경찰서수로 설정
station_totals = station_data_filtered.groupby('행정구역').size().reset_index(name='경찰서수')


# 4. CCTV 설치 현황 정리 (경제자유구역청 제외)
cctv_data_cleaned = cctv_data[['구분', '설치수량(대)']].rename(columns={'구분': '행정구역', '설치수량(대)': 'CCTV수량'})
# 경제자유구역청은 제외(원래 관할 지역은 연수구, 중구, 서구에 걸쳐 있음) 
cctv_data_cleaned = cctv_data_cleaned[~cctv_data_cleaned['행정구역'].str.contains('경제자유구역청', na=False)]


# 5. 가로등 설치 현황 정리
streetlight_total = streetlight_data.iloc[:, 1:].sum(axis=1).reset_index()
streetlight_total.columns = ['index', '가로등수']
streetlight_total['행정구역'] = streetlight_data.iloc[:, 0]  # 첫 번째 열을 행정구역 이름으로 설정
streetlight_total = streetlight_total[['행정구역', '가로등수']]

# 6. 편의점 개수 동적으로 가져오기
wd = webdriver.Chrome()

incheon_locations = ["중구", "동구", "미추홀구", "연수구", "남동구", "부평구", "계양구", "서구", "강화군", "옹진군"]

incheon_stores_data = {
    "중구": 0, "동구": 0, "미추홀구": 0, "연수구": 0,
    "남동구": 0, "부평구": 0, "계양구": 0,
    "서구": 0, "강화군": 0, "옹진군": 0
}

# -- 인천에 있는 지역 지도에 검색
for incheon_location in incheon_locations:
    wd.get(f"https://www.google.com/search?sca_esv=293021c43ebdc58d&tbm=lcl&q=인천+{incheon_location}+편의점&rflfq=1&num=10&sa=X&ved=2ahUKEwj_jKvcpY6KAxXMka8BHUmBB1AQjGp6BAgjEAE&biw=573&bih=632&dpr=1.38#rlfi=hd:;si:;")

    while True:
        html = wd.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 현재 페이지의 div요소들 모두 가져옴
        stores = soup.find_all("div", class_="VkpGBb")

        # 총 편의점 개수에 현재 페이지 편의점 수 저장
        incheon_stores_data[incheon_location] += int(len(stores))

        try:
            # 다음 페이지 버튼 누름
            next_button = wd.find_element(By.XPATH, '//*[@id="pnnext"]/span[2]')
            next_button.click()
            time.sleep(2)
        # 다음페이지 버튼 못 누를 경우 반복문 멈춤
        except:
            break
        
# -- 데이터 프레임으로 저장
stores_result = pd.DataFrame(list(incheon_stores_data.items()), columns=['행정구역', '편의점 개수'])

# 병합 기준 열 데이터 타입 통일
crime_rate['행정구역'] = crime_rate['행정구역'].astype(str)
station_totals['행정구역'] = station_totals['행정구역'].astype(str)
cctv_data_cleaned['행정구역'] = cctv_data_cleaned['행정구역'].astype(str)
streetlight_total['행정구역'] = streetlight_total['행정구역'].astype(str)
stores_result['행정구역'] = stores_result['행정구역'].astype(str)

# 6. 최종 데이터프레임 병합
final_df = crime_rate.merge(station_totals, on='행정구역', how='left')
final_df = final_df.merge(cctv_data_cleaned, on='행정구역', how='left')
final_df = final_df.merge(streetlight_total, on='행정구역', how = 'left')
final_df = final_df.merge(stores_result, on='행정구역', how = 'left')
final_df = final_df[['행정구역', '총범죄량', '총인구수', '인구수당범죄율', '경찰서수', 'CCTV수량', '가로등수', '편의점 개수']]

# 결과 저장 및 출력
final_df.to_csv('인천광역시_행정구역별_통합데이터.csv', index=False, encoding='utf-8')
final_df.tail(10)


C:\Users\mark1\AppData\Local\Temp\ipykernel_27200\3946292357.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_data_filtered['행정구역'] = station_data_filtered['주소'].str.extract(r'인천광역시\s*(\S+)')


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF765F26CC5+28821]
	(No symbol) [0x00007FF765E93850]
	(No symbol) [0x00007FF765D3578A]
	(No symbol) [0x00007FF765D0F4F5]
	(No symbol) [0x00007FF765DB6247]
	(No symbol) [0x00007FF765DCECE2]
	(No symbol) [0x00007FF765DAF0A3]
	(No symbol) [0x00007FF765D7A778]
	(No symbol) [0x00007FF765D7B8E1]
	GetHandleVerifier [0x00007FF76625FCCD+3408029]
	GetHandleVerifier [0x00007FF76627743F+3504143]
	GetHandleVerifier [0x00007FF76626B61D+3455469]
	GetHandleVerifier [0x00007FF765FEBDCB+835995]
	(No symbol) [0x00007FF765E9EB6F]
	(No symbol) [0x00007FF765E9A824]
	(No symbol) [0x00007FF765E9A9BD]
	(No symbol) [0x00007FF765E8A1A9]
	BaseThreadInitThunk [0x00007FFABF2A259D+29]
	RtlUserThreadStart [0x00007FFAC15EAF38+40]
